# EddyPro API

This notebook gives an outline for working with the API.

* First step is to define your input primary parameters.  You need to give a siteID, everything else is *optional*.

    * kwargs (dict of all optional settings):\
        * sourceDir: if not provided, assumes data are already in Outputs folder
        * dateRange (if not provided, will run all files in the current year)
        * fileType (defaults to .ghg) must specify otherwise for 
        * metadataTemplate (only required for .dat files)



In [33]:
# def makeUserConfig()


# Example run for BB on one day in 2024
siteID = 'BB'
kwargs = {
    'dateRange':['2024-04-01','2024-04-05'],
    'sourceDir':"C:/highfreq/BB/raw/",
    # 'reset':True,
    'metadataUpdates':'C:/highfreq/BB/Manual_Metadata_Updates.csv',
    'priority':'high'
}

# # # Example run for All BBS data from the Datadump Folder 20240516
# siteID = 'BBS'
# kwargs = {
#     'dateRange':['2023-06-01','2024-05-31'],
#     'sourceDir':"X:/BBS/EC_Station/2024/20240516/",
#     'searchTag':"BBS.FLUX",
#     'timeShift':30,
#     'fileType':'dat',
#     'metaDataTemplate':'Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata',
# }


# # Cospectral correction options
# hf_meth = {
#     'Moncrieff et al. (1997)':'1',
#     'Horst (1997)':'2',
#     'Ibrom et al. (2007)':'3',
#     'Fratini et al. (2012)':'4', # Recommended for closed path systems, when sufficient data (>1 month) are available
#     'Massman (2000, 2001)':'5'
# }

# if siteID == 'BBS':
#     userDefinedEddyProSettings = {
#         'Project':{
#             'hf_meth':'1'
#             },
#         'RawProcess_Settings':{
#             'v_offset':0.08,
#             'u_offset':-0.04
#             },
#         }
#     kwargs['userDefinedEddyProSettings'] = userDefinedEddyProSettings

# if 'dateRange' not in kwargs:
#     kwargs['dateRange'] = None


# Check the configurations

In [34]:
import eddyProAPI
import importlib
importlib.reload(eddyProAPI)
kwargs['queryBiometDatabase'] = True
eP = eddyProAPI.eddyProAPI(siteID,**kwargs)
eP.config['BiometUser']

Initializing tasks for BB over: ['2024-04-01 00:00' '2024-04-05 00:00']
See output: C:/temp//BB/metadata//BB_biometData_202404010000_202404050000.csv
canopy_height missing, outputting NaNs
See output: C:/temp//BB/metadata//BB_dynamicMetadata_202404010000_202404050000.csv
All tasks completed successfully


{'Biomet.net': 'C:/Biomet.net/', 'Database': 'C:/Database/'}

## Run PreProcessing

In [35]:
pre = eddyProAPI.preProcessing(siteID,**kwargs)

Initializing tasks for BB over: ['2024-04-01 00:00' '2024-04-05 00:00']
See output: C:/temp//BB/metadata//BB_biometData_202404010000_202404050000.csv
canopy_height missing, outputting NaNs
See output: C:/temp//BB/metadata//BB_dynamicMetadata_202404010000_202404050000.csv
All tasks completed successfully
Searching C:/temp//BB/raw/2024\04
[████████████████████████████████████████████████████████████] 95/95

Files Search Complete, time elapsed:  0.9815175533294678
Reading Data
Reading Complete, time elapsed:  0.05860400199890137


# Run EddyPro in Parallel over date range

In [36]:

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)

eP = eddyProAPI.runEP(siteID,**kwargs)

Initializing tasks for BB over: ['2024-04-01 00:00' '2024-04-05 00:00']
See output: C:/temp//BB/metadata//BB_biometData_202404010000_202404050000.csv
canopy_height missing, outputting NaNs
See output: C:/temp//BB/metadata//BB_dynamicMetadata_202404010000_202404050000.csv
All tasks completed successfully
Creating C:\MM_Py\EddyPro_API\temp\group_1_rp_A.eddypro for 24 files
28
Creating C:\MM_Py\EddyPro_API\temp\group_1_rp_B.eddypro for 24 files
28
Creating C:\MM_Py\EddyPro_API\temp\group_1_rp_C.eddypro for 23 files
28
Creating C:\MM_Py\EddyPro_API\temp\group_1_rp_D.eddypro for 24 files
28
Creating C:\MM_Py\EddyPro_API\temp\group_1_fcc.eddypro for 95 files
28
Initiating EddyPro Runs for group 1 on 4 cores at high priority
['C:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_A.eddypro', 'C:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_B.eddypro', 'C:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_C.eddypro', 'C:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_D.eddypro']
[██████████████████████████████████████████████████████

In [37]:
# import batchProcessing
# import importlib
# importlib.reload(batchProcessing)
# import os 
# from datetime import datetime,date

# # d_out = os.path.abspath("C:/temp/BB/eddyProAPIOutputs")
# # d_in = os.path.abspath(f"{os.getcwd()}/temp/")
# # batchProcessing.pasteWithSubprocess(d_in,d_out,option='move')
# os.rename(os.path.abspath(d_out+'/temp'),os.path.abspath(d_out+'/'+datetime.strftime(datetime.now(),format='%Y%m%d%H%M')))

# # d_in = os.path.abspath(f"{os.getcwd()}/temp/")
# # d_out = os.path.abspath("C:/temp/BB/eddyProAPIOutputs/202408021226/")

# # batchProcessing.pasteWithSubprocess(d_in,d_out)
